In [5]:
import praw
from psaw import PushshiftAPI
import datetime as dt
import pandas as pd
from psycopg2 import connect, sql

In [6]:
#PRAW Reddit Instance
reddit = praw.Reddit(
    client_id = "39xb4_RQG875f_WdFaU6nQ",
    client_secret="73AgSa_7TmG7Hy5BfgKw6J5E8uqY0Q",
    user_agent="OrlandoSucks",
)
#SQL Connect
connection = connect(
    dbname="f1sentiments",
    user="postgres",
    host="localhost",
    password="@stonM@rtin2021"
)

#Cursor Instance
Cursor = connection.cursor()

#PSAW API Instance
api = PushshiftAPI()

In [9]:
postCols = ['post_id','post_title','post_name','author','time_created','edited','is_original','is_self','locked','over_18','num_comments',
                  'score','upvote_ratio','selftext','year']
commentCols = ['author','body','body_html','created_utc','edited','comment_id','post_id','is_submitter','parent_id','score']


#postFormat = "INSERT INTO posts (post_id, post_title, post_name, author, time_created, edited, is_original, is_self, locked, over_18, num_comments, score, upvote_ratio, selftext, year) VALUES ('{}', '{}', '{}', '{}', {}, )".format()
#postFormat = "INSERT INTO posts ({0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13},{14})".format(postCols)
print(postFormat)

NameError: name 'postFormat' is not defined

In [15]:
### Get Post IDs from PushAPI ###
def query_posts_by_day(year, month, day, subreddit):
    ts_after = int(dt.datetime(year, month, day).timestamp())
    ts_before = int(dt.datetime(year, month, day+1).timestamp())

    gen = api.search_submissions(
        after=ts_after,
        before=ts_before,
        filter=['id'],
        subreddit=subreddit,
        limit=100 #for testing purposes
    )
    return gen

### Returns packet of data and submission instance for comment processing ###
def get_post_data(newId, year):
    sub_data = reddit.submission(id=newId)
    insertString = "INSERT INTO posts (post_id, post_title, post_name, author, time_created, edited, is_original, is_self, locked, over_18, num_comments, score, upvote_ratio, selftext, year) VALUES "
    values = "('{}', '{}', '{}', '{}', {}, {}, {}, {}, {}, {}, {}, {}, {}, '{}', {})".format(
        getattr(sub_data, 'id', None),
        getattr(sub_data, 'title', None),
        getattr(sub_data, 'name', None),
        getattr(getattr(sub_data, 'author', None),'name', None),
        getattr(sub_data, 'created_utc', None),
        str(getattr(sub_data, 'edited', None)).upper(),
        str(getattr(sub_data, 'is_original_content', None)).upper(),
        str(getattr(sub_data, 'is_self', None)).upper(),
        str(getattr(sub_data, 'locked', None)).upper(),
        str(getattr(sub_data, 'over_18', None)).upper(),
        getattr(sub_data, 'num_comments', None),
        getattr(sub_data, 'score', None),
        getattr(sub_data, 'upvote_ratio', None),
        getattr(sub_data, 'selftext', "NOSELFTEXT"),
        year
    )
    values = insertString + values +";"
    return [sub_data, values]

### Returns list of comment instances ###
def get_post_comments(submission, year):
    comments = submission.comments.list()
    masterList = []
    for comment in comments:
        masterList.append([
            getattr(getattr(comment, 'author', None), 'name', None),
            getattr(comment, 'body', None),
            getattr(comment, 'body_html', None),
            getattr(comment, 'created_utc', None),
            getattr(comment, 'edited', None),
            getattr(comment, 'id', None),
            getattr(comment, 'link_id', None),
            getattr(comment, 'is_submitter', None),
            getattr(comment, 'parent_id', None),
            getattr(comment, 'score', None),
            year
        ]) # Need to add year column
    return masterList
    

### Turns list of data into string separated by commas ###
def stringify_row(rowList, current, idx):
    #print(current)
    #print(idx)
    if idx <= 0:
        return current
    if idx == len(rowList):
        print(rowList)
        if isinstance(rowList[idx-1], str):
            current = "\'" + str(rowList[idx-1]) + "\'"
        else:
            current = str(rowList[idx-1])
        idx -= 2
        return stringify_row(rowList, current, idx)
    else:
        if isinstance(rowList[idx-1], str):
            current = "\'" + str(rowList[idx-1]) + "\'" + ", " + current
        else:
            current = str(rowList[idx-1]) + ", " + current
        idx -= 1
        return stringify_row(rowList, current, idx)
    
def stringify_cols(rowList, current, idx):
    #print(current)
    #print(idx)
    if idx <= 0:
        return current
    if idx == len(rowList):
        print(rowList)
        current = str(rowList[idx-1])
        idx -= 2
        return stringify_col(rowList, current, idx)
    else:
        current = str(rowList[idx-1]) + ", " + current
        idx -= 1
        return stringify_col(rowList, current, idx)

### Creates Insert SQL Statement ###
def insert_row(postRow, cols, table):
    statement = "INSERT INTO "+table+" ("+stringify_cols(cols, "", len(cols))+") VALUES(" + stringify_row(postRow, "", len(postRow)) + ");"
    execute_postgresql(statement)
    
### Executes SQL Statement passed to function ###
def execute_postgresql(statement=""):
    connection = establish_db_connection('f1sentiments','postgres','localhost','@stonM@rtin2021')
    newCursor = connection.cursor()
    print("POSTGRES SQL: Executing...\n"+str(statement))

    if statement[-1] != ";":
        print("POSTGRES SQL: NO SEMI-COLON")
    else:
        try:
            sql_obj = sql.SQL(statement)
            newCursor.execute( sql_obj )
            print("POSTGRES SQL: Finished")
        except Exception as error:
            print("POSTGRES SQL: ERROR...\n", error)
    connection.commit()
    newCursor.close()
    connection.close()

### Turns dict into list ###
def listify(dataDict):
    cols = dataDict.keys()
    output = []
    for col in cols:
        output.append(dataDict[col])
    return output

def establish_db_connection(db, username, hostname, passcode):
    #SQL Connect
    connection = connect(
        dbname=db,
        user=username,
        host=hostname,
        password=passcode
    )
    #connection.autocommit = True;
    #Connection Instance
    return connection


In [11]:
testdict = {
    'one':1,
    'two':2,
    'three':"three"
}
print(listify(testdict))

[1, 2, 'three']


In [12]:
sub_data = reddit.submission(id='2qze53')

In [13]:
packet = get_post_data('2qze53', 2014)
packet[2]

"INSERT INTO posts (post_id, post_title, post_name, author, time_created, edited, is_original, is_self, locked, over_18, num_comments, score, upvote_ratio, selftext, year) VALUES ('2qze53', 'Happy birthday to my hero, Baumgartner Zsolt!', 't3_2qze53', 'Pawy_beingretard247', 1420086804.0, FALSE, FALSE, FALSE, FALSE, FALSE, 13, 15, 0.75, '', 2014);"

In [16]:
execute_postgresql(packet[2])

POSTGRES SQL: Executing...
INSERT INTO posts (post_id, post_title, post_name, author, time_created, edited, is_original, is_self, locked, over_18, num_comments, score, upvote_ratio, selftext, year) VALUES ('2qze53', 'Happy birthday to my hero, Baumgartner Zsolt!', 't3_2qze53', 'Pawy_beingretard247', 1420086804.0, FALSE, FALSE, FALSE, FALSE, FALSE, 13, 15, 0.75, '', 2014);
POSTGRES SQL: ERROR...
 duplicate key value violates unique constraint "posts_pkey"
DETAIL:  Key (post_id)=(2qze53) already exists.



In [107]:
def getYearsData(year):
    days = [31,28,31,30,31,30,31,31,30,31,30,31]
    months = [1,2,3,4,5,6,7,8,9,10,11,12]
    if year is in [2012,2016,2020]:
        days[1] = 29
    
    for month in months:
        for day in range(days[month]+1):
            gen = query_posts_by_day(year, month, day, 'formula1')
            for sub in gen:
                # Getting Post Data Packet
                packet = get_post_data(sub._d, year)
                postData = packet[0] # Post Data
                submission = packet[1] # Submission Instance
                
                #Sending Post Data
                execute_postgresql(None, insert_row(postData, 'posts'))
                
                

SyntaxError: invalid syntax (Temp/ipykernel_15780/1489583867.py, line 4)

In [108]:
"('{post_id}')".format(post_id="zgkerno")

"('zgkerno')"

In [111]:
test = False
print(str(test).upper())

FALSE
